In [51]:
import pandas as pd 
import numpy as np
from matplotlib import style
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#tdqm = progress bar
from tqdm import tqdm
from datetime import datetime
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn import utils

In [52]:
# Number of rows
nrows = 10000000
# Chunksize = 10000 
chunksize = 10000
# Vitou's path
# path = 'C:/Users/sirus/Downloads/train.csv'
# Muna's path
path = '/Users/muna/Development/DataScience/new-york-city-taxi-fare-prediction/train.csv'
# Columns to read from the data
# Todo :split date into meaningful data
# cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount']
cols = ['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount']

#list to hold the batch dataframe
df_list = []

for df_chunk in tqdm(pd.read_csv(path,usecols=cols, chunksize=chunksize,nrows=nrows)):
    df_list.append(df_chunk)
    
# Merge all dataframes into one dataframe
data = pd.DataFrame()
data = pd.concat(df_list)
# # Delete the dataframe list to release memory
del df_list, df_chunk

1000it [00:18, 53.30it/s]


### Let's take a look at the data

In [53]:
%%time
# data.describe(include='all')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 25 µs


### Check for missing values

In [54]:
%%time
# Checking for missing values
data.isnull().sum()

CPU times: user 1.07 s, sys: 457 ms, total: 1.53 s
Wall time: 958 ms


fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    69
dropoff_latitude     69
passenger_count       0
dtype: int64

### From our observation, there are some missing values, so we remove them

In [55]:
%%time
data = data.dropna(how = 'any', axis = 'rows')

CPU times: user 1.48 s, sys: 367 ms, total: 1.85 s
Wall time: 1.49 s


### Let's take a look at the data

In [56]:
data['fare_amount'].describe()

count    9.999931e+06
mean     1.133849e+01
std      9.799845e+00
min     -1.077500e+02
25%      6.000000e+00
50%      8.500000e+00
75%      1.250000e+01
max      1.273310e+03
Name: fare_amount, dtype: float64

### From the description above, some of the fare_amount have negative values

We are going to remove all negatvie fare_amounts

In [57]:
%%time
data = data[(data['fare_amount'] > 0)]

CPU times: user 495 ms, sys: 188 ms, total: 683 ms
Wall time: 686 ms


In [58]:
data['fare_amount'].describe()

count    9.999242e+06
mean     1.133966e+01
std      9.798609e+00
min      1.000000e-02
25%      6.000000e+00
50%      8.500000e+00
75%      1.250000e+01
max      1.273310e+03
Name: fare_amount, dtype: float64

In [59]:
data['fare_amount'].describe()

count    9.999242e+06
mean     1.133966e+01
std      9.798609e+00
min      1.000000e-02
25%      6.000000e+00
50%      8.500000e+00
75%      1.250000e+01
max      1.273310e+03
Name: fare_amount, dtype: float64

### Let's take a detailed look at the passenger count

In [60]:
data['passenger_count'].describe()

count    9.999242e+06
mean     1.684807e+00
std      1.323424e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      2.080000e+02
Name: passenger_count, dtype: float64

The highest passenger count per taxi ride is 208, which is not possible. So we remove all passenger_count grater than 6, since 6 is the maximum passenger capacity for Uber/Lyft if the ride is an SUV

In [61]:
data = data[(data['passenger_count'] <= 6)]

In [62]:
data['passenger_count'].describe()

count    9.999225e+06
mean     1.684595e+00
std      1.308064e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      6.000000e+00
Name: passenger_count, dtype: float64

Now the highest passenger capacity is 6

Next we remove all passenger count that are zero

In [63]:
data = data[(data['passenger_count'] > 0)]

In [64]:
data['passenger_count'].describe()

count    9.963965e+06
mean     1.690557e+00
std      1.306525e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      6.000000e+00
Name: passenger_count, dtype: float64

### Looking at the pickup|droppoff latitudes and longitudes

In [65]:
data[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].describe()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,9.963965e+06,9.963965e+06,9.963965e+06,9.963965e+06
mean,3.991903e+01,-7.250735e+01,3.991906e+01,-7.250880e+01
std,9.333588e+00,1.300399e+01,9.247605e+00,1.286803e+01
min,-3.492264e+03,-3.439245e+03,-3.488080e+03,-3.426601e+03
25%,4.073491e+01,-7.399208e+01,4.073403e+01,-7.399139e+01
50%,4.075263e+01,-7.398181e+01,4.075315e+01,-7.398015e+01
75%,4.076712e+01,-7.396710e+01,4.076810e+01,-7.396367e+01
max,3.344459e+03,3.457626e+03,3.351403e+03,3.457622e+03


Looking at the info above, the maximum pickup_latitude/pickup_longtide/droppoff_latitude/dropoff_longitude are over 3000 WITH their respective minimum values are over -3000

Latitudes range from -90 to 90 while longitudes range from -180 to 180, for single degree format. So we remove all values that are not with the latitude and longitude ranges

In [66]:
data = data.drop((data[(data['pickup_latitude'] > 90) | (data['pickup_latitude'] < -90)]).index, axis=0)

In [67]:
data = data.drop((data[(data['pickup_longitude'] > 180) | (data['pickup_longitude'] < -180)]).index, axis=0)

We will do the same for dropoff cordinates

In [68]:
data = data.drop((data[(data['dropoff_latitude'] > 90) | (data['dropoff_latitude'] < -90)]).index, axis=0)

In [69]:
data = data.drop((data[(data['dropoff_longitude'] > 180) | (data['dropoff_longitude'] < -180)]).index, axis=0)

In [70]:
data[['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']].describe()

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,9.963490e+06,9.963490e+06,9.963490e+06,9.963490e+06
mean,3.991535e+01,-7.249567e+01,3.991792e+01,-7.250012e+01
std,6.125685e+00,1.046701e+01,6.118556e+00,1.044907e+01
min,-7.482416e+01,-1.686035e+02,-7.419320e+01,-1.733420e+02
25%,4.073491e+01,-7.399207e+01,4.073403e+01,-7.399139e+01
50%,4.075263e+01,-7.398181e+01,4.075315e+01,-7.398015e+01
75%,4.076712e+01,-7.396710e+01,4.076810e+01,-7.396367e+01
max,8.974216e+01,1.541008e+02,8.155467e+01,1.541008e+02


### Feature Engineering

In [71]:
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(data)

### In further observation, we have pickup_longitude, puckup_latitiude, dropoff_longitude and dropoff_latitude. We can calculate the distance

In [72]:
# Function that calculates distance between pickup location and dropoff location
def getDistance(lat1,lon1,lat2,lon2):
    r = 6378 # earth's radius
    lat1 = np.deg2rad(lat1)
    lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2)
    lon2 = np.deg2rad(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = r*c
    
    return distance



In [73]:
%%time
# Add new cloumn "distance" to the data
data['distance'] = getDistance(data.pickup_latitude, data.pickup_longitude, 
                                      data.dropoff_latitude, data.dropoff_longitude)

CPU times: user 2.65 s, sys: 548 ms, total: 3.19 s
Wall time: 1.4 s


In [74]:
data['distance'].describe()

count    9.963490e+06
mean     1.956326e+01
std      3.680907e+02
min      0.000000e+00
25%      1.214020e+00
50%      2.118768e+00
75%      3.879586e+00
max      1.286588e+04
Name: distance, dtype: float64

We have some distances which are zero. so we remove all

In [75]:
data = data[(data['distance'] > 0)]

In [76]:
data['distance'].describe()

count    9.678609e+06
mean     2.013909e+01
std      3.734531e+02
min      8.384762e-05
25%      1.280997e+00
50%      2.183518e+00
75%      3.964757e+00
max      1.286588e+04
Name: distance, dtype: float64

In [77]:
%%time
print(data[::10])

         fare_amount          pickup_datetime  pickup_longitude  \
0                4.5  2009-06-15 17:26:21 UTC        -73.844311   
10               5.3  2012-04-08 07:30:50 UTC        -73.996335   
22               4.5  2009-08-06 18:17:23 UTC        -73.991707   
33               5.7  2011-09-07 14:05:00 UTC        -73.976162   
43              12.1  2009-06-10 21:28:00 UTC        -73.988558   
...              ...                      ...               ...   
9999950          9.7  2009-05-27 08:38:58 UTC        -74.005170   
9999961          8.9  2010-02-14 00:14:13 UTC        -73.983248   
9999971          5.0  2014-01-10 12:32:00 UTC        -73.992987   
9999981          4.9  2010-06-16 09:21:00 UTC        -73.696362   
9999991         12.5  2013-09-13 07:56:00 UTC        -73.781852   

         pickup_latitude  dropoff_longitude  dropoff_latitude  \
0              40.721319         -73.841610         40.712278   
10             40.737142         -73.980721         40.733559   


### Creating new features like year, month, day, hour and dayOfWeek from pickup_datetime

In [78]:
def split_datetime(df):
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format="%Y-%m-%d %H:%M:%S UTC")
        df['year'] = df['pickup_datetime'].dt.year
        df['month'] = df['pickup_datetime'].dt.month
        df['day'] = df['pickup_datetime'].dt.day
        df['hour'] = df['pickup_datetime'].dt.hour
        df['dayOfWeek'] = df['pickup_datetime'].dt.dayofweek
        
        return df

In [79]:
%%time
split_datetime(data)

CPU times: user 34.8 s, sys: 489 ms, total: 35.3 s
Wall time: 34.9 s


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,distance,year,month,day,hour,dayOfWeek
0,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,0.002701,0.009041,1.031896,2009,6,15,17,0
1,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,0.036780,0.070701,8.459418,2010,1,5,16,1
2,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,0.008504,0.010708,1.391052,2011,8,18,0,3
3,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,0.004437,0.024949,2.802346,2012,4,21,4,5
4,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,0.011440,0.015754,2.001353,2010,3,9,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,5.7,2012-08-12 01:18:00,-73.999464,40.728452,-73.993299,40.742100,2,0.006165,0.013648,1.605786,2012,8,12,1,6
9999996,5.5,2013-08-07 10:28:00,-73.968467,40.759367,-73.964967,40.769027,1,0.003500,0.009660,1.115078,2013,8,7,10,2
9999997,14.0,2013-10-29 08:29:00,-73.997952,40.733717,-73.973448,40.759122,5,0.024504,0.025405,3.502599,2013,10,29,8,1
9999998,10.5,2012-04-07 16:41:33,-73.992700,40.752021,-73.964705,40.772849,1,0.027995,0.020828,3.308606,2012,4,7,16,5


Now that we have split pickup_datetime into year, month, day, hour and year

### Let's take a look at hour adn dayOfWeek

In [80]:
%%time
data['hour'].describe()

CPU times: user 414 ms, sys: 63.9 ms, total: 478 ms
Wall time: 293 ms


count    9.678609e+06
mean     1.351339e+01
std      6.515992e+00
min      0.000000e+00
25%      9.000000e+00
50%      1.400000e+01
75%      1.900000e+01
max      2.300000e+01
Name: hour, dtype: float64

According to New York Times, rush hours is ususally between 7 to 9AM and from 4 to 6PM. Let's create a new feature from hour

In [81]:
# 1 for rush hour, 0 for not
def rush_hour(hour):
    if hour in range(7, 10) or hour in range(4, 7):
        return 1
    else:
        return 0

In [82]:
%%time
data['rush_hour'] = data['hour'].apply(rush_hour)

CPU times: user 7.06 s, sys: 179 ms, total: 7.24 s
Wall time: 7.52 s


In [83]:
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,distance,year,month,day,hour,dayOfWeek,rush_hour
count,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06
mean,1.134274e+01,-7.383489e+01,4.065208e+01,-7.383947e+01,4.065473e+01,1.690974e+00,1.679034e-01,9.580614e-02,2.013909e+01,2.011746e+03,6.267631e+00,1.571740e+01,1.351339e+01,3.041449e+00,1.699693e-01
std,9.670004e+00,3.566599e+00,2.875887e+00,3.510276e+00,2.859540e+00,1.306586e+00,3.260679e+00,1.733879e+00,3.734531e+02,1.866468e+00,3.435410e+00,8.684960e+00,6.515992e+00,1.949078e+00,3.756058e-01
min,1.000000e-02,-1.686035e+02,-7.482416e+01,-1.733420e+02,-7.419320e+01,1.000000e+00,0.000000e+00,0.000000e+00,8.384762e-05,2.009000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399228e+01,4.073645e+01,-7.399158e+01,4.073545e+01,1.000000e+00,6.329000e-03,7.141113e-03,1.280997e+00,2.010000e+03,3.000000e+00,8.000000e+00,9.000000e+00,1.000000e+00,0.000000e+00
50%,8.500000e+00,-7.398211e+01,4.075330e+01,-7.398060e+01,4.075382e+01,1.000000e+00,1.288000e-02,1.438100e-02,2.183518e+00,2.012000e+03,6.000000e+00,1.600000e+01,1.400000e+01,3.000000e+00,0.000000e+00
75%,1.250000e+01,-7.396835e+01,4.076750e+01,-7.396538e+01,4.076838e+01,2.000000e+00,2.421000e-02,2.753000e-02,3.964757e+00,2.013000e+03,9.000000e+00,2.300000e+01,1.900000e+01,5.000000e+00,0.000000e+00
max,9.520000e+02,7.393784e+01,8.974216e+01,7.394509e+01,8.155467e+01,6.000000e+00,1.194450e+02,1.155504e+02,1.286588e+04,2.015000e+03,1.200000e+01,3.100000e+01,2.300000e+01,6.000000e+00,1.000000e+00


In [84]:
%%time
data['dayOfWeek'].describe()

CPU times: user 364 ms, sys: 91.9 ms, total: 456 ms
Wall time: 286 ms


count    9.678609e+06
mean     3.041449e+00
std      1.949078e+00
min      0.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      5.000000e+00
max      6.000000e+00
Name: dayOfWeek, dtype: float64

We can also create a new feature from daysOfWeek. We can check if it's a weekend or not

In [85]:
# 1 for weekend, 0 for not
def weekend(dayOfWeek):
    if dayOfWeek == 0 or dayOfWeek == 6:
        return 1
    else:
        return 0

In [86]:
%%time
data['weekend'] = data['dayOfWeek'].apply(weekend)

CPU times: user 2.97 s, sys: 143 ms, total: 3.12 s
Wall time: 3.1 s


In [87]:
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,distance,year,month,day,hour,dayOfWeek,rush_hour,weekend
count,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06
mean,1.134274e+01,-7.383489e+01,4.065208e+01,-7.383947e+01,4.065473e+01,1.690974e+00,1.679034e-01,9.580614e-02,2.013909e+01,2.011746e+03,6.267631e+00,1.571740e+01,1.351339e+01,3.041449e+00,1.699693e-01,2.593431e-01
std,9.670004e+00,3.566599e+00,2.875887e+00,3.510276e+00,2.859540e+00,1.306586e+00,3.260679e+00,1.733879e+00,3.734531e+02,1.866468e+00,3.435410e+00,8.684960e+00,6.515992e+00,1.949078e+00,3.756058e-01,4.382742e-01
min,1.000000e-02,-1.686035e+02,-7.482416e+01,-1.733420e+02,-7.419320e+01,1.000000e+00,0.000000e+00,0.000000e+00,8.384762e-05,2.009000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399228e+01,4.073645e+01,-7.399158e+01,4.073545e+01,1.000000e+00,6.329000e-03,7.141113e-03,1.280997e+00,2.010000e+03,3.000000e+00,8.000000e+00,9.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,8.500000e+00,-7.398211e+01,4.075330e+01,-7.398060e+01,4.075382e+01,1.000000e+00,1.288000e-02,1.438100e-02,2.183518e+00,2.012000e+03,6.000000e+00,1.600000e+01,1.400000e+01,3.000000e+00,0.000000e+00,0.000000e+00
75%,1.250000e+01,-7.396835e+01,4.076750e+01,-7.396538e+01,4.076838e+01,2.000000e+00,2.421000e-02,2.753000e-02,3.964757e+00,2.013000e+03,9.000000e+00,2.300000e+01,1.900000e+01,5.000000e+00,0.000000e+00,1.000000e+00
max,9.520000e+02,7.393784e+01,8.974216e+01,7.394509e+01,8.155467e+01,6.000000e+00,1.194450e+02,1.155504e+02,1.286588e+04,2.015000e+03,1.200000e+01,3.100000e+01,2.300000e+01,6.000000e+00,1.000000e+00,1.000000e+00


rush_hour and weekend are categorical either 1 or 0, so we can crerate dummy columns from them

In [88]:
data = pd.get_dummies(data, columns=['rush_hour','weekend'])

In [89]:
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,distance,year,month,day,hour,dayOfWeek,rush_hour_0,rush_hour_1,weekend_0,weekend_1
count,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06,9.678609e+06
mean,1.134274e+01,-7.383489e+01,4.065208e+01,-7.383947e+01,4.065473e+01,1.690974e+00,1.679034e-01,9.580614e-02,2.013909e+01,2.011746e+03,6.267631e+00,1.571740e+01,1.351339e+01,3.041449e+00,8.300307e-01,1.699693e-01,7.406569e-01,2.593431e-01
std,9.670004e+00,3.566599e+00,2.875887e+00,3.510276e+00,2.859540e+00,1.306586e+00,3.260679e+00,1.733879e+00,3.734531e+02,1.866468e+00,3.435410e+00,8.684960e+00,6.515992e+00,1.949078e+00,3.756058e-01,3.756058e-01,4.382742e-01,4.382742e-01
min,1.000000e-02,-1.686035e+02,-7.482416e+01,-1.733420e+02,-7.419320e+01,1.000000e+00,0.000000e+00,0.000000e+00,8.384762e-05,2.009000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399228e+01,4.073645e+01,-7.399158e+01,4.073545e+01,1.000000e+00,6.329000e-03,7.141113e-03,1.280997e+00,2.010000e+03,3.000000e+00,8.000000e+00,9.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.500000e+00,-7.398211e+01,4.075330e+01,-7.398060e+01,4.075382e+01,1.000000e+00,1.288000e-02,1.438100e-02,2.183518e+00,2.012000e+03,6.000000e+00,1.600000e+01,1.400000e+01,3.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,1.250000e+01,-7.396835e+01,4.076750e+01,-7.396538e+01,4.076838e+01,2.000000e+00,2.421000e-02,2.753000e-02,3.964757e+00,2.013000e+03,9.000000e+00,2.300000e+01,1.900000e+01,5.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
max,9.520000e+02,7.393784e+01,8.974216e+01,7.394509e+01,8.155467e+01,6.000000e+00,1.194450e+02,1.155504e+02,1.286588e+04,2.015000e+03,1.200000e+01,3.100000e+01,2.300000e+01,6.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [90]:
# sns.barplot(x=data['year'],y=data["fare_amount"],data=data).set_title("Fare Amount over Years")

### Looks like fares have been incrasing over the years

In [91]:
# sns.barplot(x=data['hour'],y=data["fare_amount"],data=data).set_title("Pickup hour vs fare amount")

#### The fare amount is highest around 5am

In [92]:
# sns.barplot(x=data['dayOfWeek'],y=data["fare_amount"],data=data).set_title("Pickup days vs fare amount")

### Correlation between features/variables

In [93]:
# %%time
# correlation= data.corr()
# colormap = plt.cm.inferno
# mask = np.array(correlation)
# mask[np.tril_indices_from(mask)] = False
# fig=plt.gcf()
# fig.set_size_inches(30,12)
# sns.heatmap(data=correlation ,mask=mask,square=True,annot=True,cbar=True,cmap=colormap, linecolor='White', linewidths=0.1)

In [94]:
%%time
feature_cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count',
                'abs_diff_longitude','abs_diff_latitude','distance','year','month','day','hour','dayOfWeek',
                'rush_hour_0','rush_hour_1','weekend_0','weekend_1']

X = data[feature_cols] 
y = data['fare_amount']

CPU times: user 341 ms, sys: 287 ms, total: 628 ms
Wall time: 591 ms


In [95]:
from sklearn.model_selection import train_test_split

In [96]:
%%time
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.35,random_state=6)

CPU times: user 5.23 s, sys: 1.5 s, total: 6.73 s
Wall time: 8.18 s


In [97]:
%%time
lr = LinearRegression()
lr.fit(X_train,y_train)

CPU times: user 6.1 s, sys: 2.84 s, total: 8.93 s
Wall time: 8.62 s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [98]:
%%time
lr_predict = lr.predict(X_test)
mse = metrics.mean_squared_error(y_test, lr_predict)
rmse = np.sqrt(mse)
print(f'RMSE of Logistic Regresion: {rmse}')

RMSE of Logistic Regresion: 9.57562048157971
CPU times: user 720 ms, sys: 869 ms, total: 1.59 s
Wall time: 1.19 s


In [99]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=10,bootstrap=True,random_state=3)
rf.fit(X_train,y_train)

y_test_pred = rf.predict(X_test)

y_train_err = metrics.mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(y_train_err)
print(f'RMSE of Logistic Regresion: {rmse}')